### Connect google drive

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

### Clone, configure & compile Darknet



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Compile, Warnings are fine
!make

### Configure yolov3.cfg file
change `max_batches` and `classes` acordingly.


In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

### Create .names and .data files

In [ ]:
!echo -e 'Without Mask\nWith Mask\nMask worn incorrect' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

### Save yolov3_training.cfg and obj.names files in Google drive

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

### Create a folder and unzip image train dataset

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images_train.zip -d data/obj

### Create train.txt file

In [ ]:
import glob
import random
images_list = glob.glob("data/obj/test/*.png")

# Uncomment to get random train/test split (Only use if no retraining/evaluation ect.)
# k = int(len(images_list) * 0.8)
# print(len(images_list))
# train = random.sample(images_list, k=k)
# print(len(train))
# test = list(set(images_list) - set(train))
# print( len(test))

# with open("data/train.txt", "w") as f:
#     f.write("\n".join(train))
# with open("data/test.txt", "w") as f:
#     f.write("\n".join(test))

with open("data/train.txt", "w") as f:
     f.write("\n".join(images_list))

### Download pre-trained weights for the convolutional layers file
Only needed when training without previous training.

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### Start training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

### Continue Training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show